# Real case: Underevaluated center back for a 3rd division french club 

---

### Objective
Identify an **undervalued center-back** who excels in aerial duels and defensive reliability, but remains off the radar

---

### Focus

We will follow this major criteria:

- The player salary should be less than 70% of median player in National 
- Search for **aerial** dominant players
- Low error leading to a goal conceded is a strong bonus
- High % successful pass is a bonus
- Young player is a bonus
- No age restriction, should be a starting player candidate 

In [94]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Charger les données nettoyées
df = pd.read_csv('../data/processed/cleaned_data.csv')

# Filter on the salary limit fixed by the club

In [95]:
# Get National players
national_players = df[df['Division']=='Championnat National']

# Calculate the median salary for a player in National
median_salary_national = national_players['Wage'].median()

# Club limit: 70% of median salary
club_limit = 0.7*median_salary_national
print('Club limit: ', club_limit)

# Filter on the salary limit
national_players_filtered = national_players[national_players['Wage'] <= club_limit]

# Create the dataframe with the players that are under the club limit, from every division
df_wage_filtered = df[(df['Wage'] <= club_limit) & (df['Position'].str.contains('D \\(C\\)'))]

# Display the number of players in the dataframe
print('Number of players in the dataframe: ', len(df_wage_filtered))


Club limit:  568.75
Number of players in the dataframe:  183


# Find out aerial dominant players

In [96]:

# Get the top 50 players with the highet aerial duels win rate
top_50_aerial_duels_win_rate = df_wage_filtered.sort_values(by='Headers Won/90', ascending=False).head(50)

# Get the top 50 players with the most aerial duels per 90
top_50_aerial_duels_per_90 = df_wage_filtered.sort_values(by='Aerial Duels/90', ascending=False).head(50)

# Find players that are in the both top lists
common_players = top_50_aerial_duels_win_rate[top_50_aerial_duels_win_rate['ID'].isin(top_50_aerial_duels_per_90['ID'])]

# Display the resulting dataframe
print('Number of players in both top 50 lists:', len(common_players))
common_players.head()

# Redo it by selecting the top 40 players, not 50
top_40_aerial_duels_win_rate = df_wage_filtered.sort_values(by='Headers Won/90', ascending=False).head(40)
top_40_aerial_duels_per_90 = df_wage_filtered.sort_values(by='Aerial Duels/90', ascending=False).head(40)

common_players_40 = top_40_aerial_duels_win_rate[top_40_aerial_duels_win_rate['ID'].isin(top_40_aerial_duels_per_90['ID'])]

# Display the resulting dataframe
print('Number of players in both top 40 lists:', len(common_players_40))



Number of players in both top 50 lists: 45
Number of players in both top 40 lists: 36


# Focus on low error leading to a goal players 

In [97]:
# Calculate the median 